In [ ]:
import sys
from timeit import timeit

sys.path.append('./src/')
from matrix import *

n = 64
m = 3
p = 5

A_squared = Matrix.random(n,n)
B_squared = Matrix.random(n,n)

A_not_squared = Matrix.random(m,n)
B_not_squared = Matrix.random(n,p)

## Matrix Multiplication

1. Implement the `strassen_matrix_mult` function to multiply two $2^{n} \times 2^{n}$ matrices by using the Strassen's algorithm;  
  

2. Generalize `strassen_matrix_mult` to deal with any kind of matrix pair that can be multiplied (possibly also non-square matrices) and prove that the asymptotic complexity does not change; 
  

3. Improve the implementation of the function by reducing the number of auxiliary matrices and test the effects on the execution time;  
  

4. Answer to the following question: how much is the minimum auxiliary
space required to evaluate the Strassen’s algorithm? Motivate the answer.

##### 
1. 

In [ ]:
def strassen_matrix_mult(A: Matrix, B: Matrix) -> Matrix:
    
    # Basic case
    if max(A.num_of_rows, B.num_of_cols, A.num_of_cols) < 32:
        return gauss_matrix_mult(A, B)

    # Recursive
    A11, A12, A21, A22 = get_matrix_quadrants(A)
    B11, B12, B21, B22 = get_matrix_quadrants(B) 

    # First batch of sums Theta(n^2)
    S1 = B12 - B22
    S2 = A11 + A12
    S3 = A21 + A22
    S4 = B21 - B11
    S5 = A11 + A22
    S6 = B11 + B22
    S7 = A12 - A22
    S8 = B21 + B22
    S9 = A11 - A21
    S10 = B11 + B12
    
    # Recursive calls
    P1 = strassen_matrix_mult(A11, S1)
    P2 = strassen_matrix_mult(S2, B22)
    P3 = strassen_matrix_mult(S3, B11)
    P4 = strassen_matrix_mult(A22, S4)
    P5 = strassen_matrix_mult(S5, S6)
    P6 = strassen_matrix_mult(S7,S8)
    P7 = strassen_matrix_mult(S9, S10)

    # Second batch of sums Theta(n^2)
    C11 = P5 + P4 - P2 + P6
    C12 = P1 + P2
    C21 = P3 + P4
    C22 = P5 + P1 - P3 - P7

    # Build the resault matrix
    result = Matrix([[0 for x in range(B.num_of_cols)]
                    for y in range(A.num_of_rows)], clone_matrix=False)

    # Copying Cij into the resault matrix            
    result.assign_submatrix(0, 0, C11)
    result.assign_submatrix(0, result.num_of_cols//2, C12)
    result.assign_submatrix(result.num_of_rows//2, 0, C21)
    result.assign_submatrix(result.num_of_rows//2, 
                            result.num_of_cols//2, C22)

    return result

In [ ]:
C = strassen_matrix_mult(A_squared, B_squared)
print(C)

#####   
2.

The `generalized_strassen_matrix_mult` can be applied to any pair of matrices that can be multiplied. In order to achieve that, it performs a *zero padding* (i.e it modifies the matrix dimensions, by adding zeros) to make the matrices $2^n$ squared and then it calls the `strassen_matrix_mult` on them. At the end, by getting rid of the extra zeros from the product, it fixes the sizes and it returns the result. For the padding we need, for both matrices, two nested `for loops` from `0` to `n`, one to create a zero matrix and the other, inside the function `.assign_submatrix()`, used to put the original matrix into its zero wrapper, resulting in a total complexity of $4 \Theta(n^2)$. Then we apply the `strassen_matrix_mult` having complexity of $ \Theta(n^{log_2(7)})$ and finally with the function `.submatrix()` we fix the sizes with another nested `for loop` from `0` to `n` having complexity of $\Theta(n^2)$. Summing all the contributes we have the overall complexity:  
$$ \\ 4\Theta(n^2) + \Theta(n^{log_2(7)}) + \Theta(n^2) = 
\Theta(n^{log_2(7)}) + 5\Theta(n^2)
\simeq \Theta(n^{log_2(7)}) < \Theta(n^{3}) \\ $$ 
 
which is the same as the one of the `strassen_matrix_mult`.  A possible implementation could be:

In [ ]:
def generalized_strassen_matrix_mult(A: Matrix, B: Matrix) -> Matrix:
    
    # Determinating the size of enlarged matrices
    n = max(A.num_of_rows, B.num_of_cols, A.num_of_cols)
    n = 2**math.ceil(math.log2(n))

    # Zero padding to enlarge the matrix A
    A_padded = Matrix([[0 for x in range(n)]
                    for y in range(n)], clone_matrix=False)
    A_padded.assign_submatrix(0, 0, A)

    # Zero padding to enlarge the matrix B
    B_padded = Matrix([[0 for x in range(n)]
                    for y in range(n)], clone_matrix=False)
    B_padded.assign_submatrix(0, 0, B)

    # Standard strassen and fixing the size
    return strassen_matrix_mult(A_padded, B_padded).submatrix(0, A.num_of_rows, 0, B.num_of_cols)

In [ ]:
C = generalized_strassen_matrix_mult(A_not_squared, B_not_squared)
print(C)

#####  
3.


We are going to take in consideration the multiplication of two $2^n$ squared matrices, since the generalized case will fall into this, as shown previously. In order to reduce the numer of auxiliary matrices it should be noticed that the recursion is used just in a set of equations (i.e the ones computing *$P_k$* ) and that at most two matrices, *$S_i$* and *$S_j$* , are used per time. Since all the other equations are given by sums and subtractions, we can split them without risking to increase the number of recurvise calls. Considering also that the majority of the matrices are not reused in more then one equation we can decrease the total amount of auxiliary matrices to three, two *$S$* matrices and one *$P$* matrix as follows:


In [ ]:
def strassen_matrix_mult_optimised(A: Matrix, B: Matrix) -> Matrix:
    
    # Basic case
    if max(A.num_of_rows, B.num_of_cols, A.num_of_cols) < 32:
        return gauss_matrix_mult(A, B)

    # Recursive
    A11, A12, A21, A22 = get_matrix_quadrants(A)
    B11, B12, B21, B22 = get_matrix_quadrants(B) 
    
    # Recursive calls with overwriting
    # New variables = Old Variables
    
    #S1 = S1, P = P1
    S1 = B12 - B22
    P = strassen_matrix_mult(A11, S1)
    C12 = P
    C22 = P
    
    #S1 = S4, P = P4
    S1 = B21 - B11
    P = strassen_matrix_mult(A22, S1)
    C11 = P
    C21 = P
    
    #S1 = S2, P = P2
    S1 = A11 + A12
    P = strassen_matrix_mult(S1, B22)
    C12 += P
    C11 -= P
    
    #S1 = S3, P = P3
    S1 = A21 + A22
    P = strassen_matrix_mult(S1, B11)
    C21 += P
    C22 -= P
    
    #S1 = S5, S2 = S6, P = P5
    S1 = A11 + A22
    S2 = B11 + B22
    P = strassen_matrix_mult(S1, S2)
    C11 += P
    C22 += P
    
    
    #S1 = S7, S2 = S8, P = P6
    S1 = A12 - A22
    S2 = B21 + B22
    P = strassen_matrix_mult(S1,S2)
    C11 += P
    
    
    #S1 = S9, S2 = S10, P = P7
    S1 = A11 - A21
    S2 = B11 + B12
    P = strassen_matrix_mult(S1, S2)
    C22 -= P
    
    
    # Build the resault matrix
    result = Matrix([[0 for x in range(B.num_of_cols)]
                    for y in range(A.num_of_rows)], clone_matrix=False)

    # Copying Cij into the resault matrix            
    result.assign_submatrix(0, 0, C11)
    result.assign_submatrix(0, result.num_of_cols//2, C12)
    result.assign_submatrix(result.num_of_rows//2, 0, C21)
    result.assign_submatrix(result.num_of_rows//2, 
                            result.num_of_cols//2, C22)

    return result

In [ ]:
C = strassen_matrix_mult_optimised(A_squared, B_squared)
print(C)

In order to test the performances of `strassen_matrix_mult_optimised` we can use the following piece of code:

In [ ]:
from sys import stdout
seed(0)

for i in range(5,10):
    size = 2**i
    stdout.write(f'{size}')
    A = Matrix.random(size, size)
    B = Matrix.random(size, size)

    for funct in ['strassen_matrix_mult', 'strassen_matrix_mult_optimised']:
        T = timeit(f'{funct}(A,B)', globals=locals(), number=1)
        stdout.write('\t{:.3f}'.format(T))
        stdout.flush()
    stdout.write('\n')

Which will produce something similar to:

![performance](img/perfomance.png)

and as it can be seen from the graph the optimised version shows some benefits when the size of the matrices is $\geq 2^{10}$

#####  
4.

Recalling the recursion tree and taking into account that the tree is bounded at $log_2(32) = 5$ from below, the number of calls to `strassen_matrix_mult` is given by $ \sum_{i = 0}^{log_2(n) - 5 - 1} 7^i $ , where the extra $-1$ in the upper bound is given due to the implementation with `<` instead of `<=`. At each call, three auxiliary matrices having half of the dimension with respect to the previous call, are made. Since we are interested in the minimum space required, we have to evaluate 
$$ \sum \text{#auxiliary_matrices} \times \text{#calls} \times \text{matrix_size}  \\ $$ 
which in formal terms become:

$$
 \left.
  \begin{cases}
    \sum_{i = 0}^{log_2(n) - 6} 3 \times 7^i \times {({{n} \over {2^{i+1}}})^2} =
{{3 n^2} \over {4}} \times  \sum_{i = 0}^{log_2(n) - 6}  {({{7} \over {4}})^i} =
{{3n^2} \over {4}} {{1+{{{7} \over {4}}^{log_2(n) - 5}}} \over {{3} \over {4}}} =
n^2 (1 + {{7 \over 4}^{log_2(n) - 5}})& \text{when } n > 32 \\
0 & \text{otherwise } 
  \end{cases}
  \right. 
$$
  
and since $n$ is a power of $2$ the minimum space required will be an integer as expected.